## BigQuery Agent

#### Imports 

In [ ]:
#!pip install pybigquery

In [ ]:
# Langchain specific imports
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents import AgentExecutor
from langchain.llms.openai import OpenAI
import langchain

# Other imports
from sqlalchemy.engine import create_engine
from urllib.parse import quote_plus
from google.cloud import bigquery
import pkg_resources
import sqlalchemy
import logging
import pyodbc

import os 

##### Setup logging

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

##### Log dependencies 

In [ ]:
logger.info(f'Using langchain=={langchain.__version__}')
logger.info(f"Using pyodbc=={pkg_resources.get_distribution('pyodbc').version}")
logger.info(f'Using sqlalchemy=={sqlalchemy.__version__}')


#### Essentials

In order to use LangChain or PyODBC to talk to databases directly - ensure the correct and respective drivers are installed in your work machine. 

For MS SQL:
https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/install-microsoft-odbc-driver-sql-server-macos?view=sql-server-ver15 <br><br>

For BigQuery:<br>
-- Landing Page<br>
https://cloud.google.com/bigquery/docs/reference/odbc-jdbc-drivers<br>
-- Install ODBC release 2.5.2.1004<br>
https://cloud.google.com/bigquery/docs/reference/odbc-jdbc-drivers#odbc_release_2521004<br>
-- Congifuration Guide <br>
https://storage.googleapis.com/simba-bq-release/odbc/Simba%20Google%20BigQuery%20ODBC%20Connector%20Install%20and%20Configuration%20Guide-2.5.2.1004.pdf<br>



In [ ]:
PROJECT = 'arun-genai-bb'
LOCATION = 'us-central1'
MODEL_NAME = 'chat-bison'
DATASET = 'hotel_reservations'
SERVICE_ACCOUNT_KEY_PATH = './credentials/vai-key.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_KEY_PATH
SERVICE_ACCOUNT_KEY_PATH_ABS = os.path.abspath(SERVICE_ACCOUNT_KEY_PATH)
SERVICE_ACCOUNT_EMAIL = 'as-service@arun-genai-bb.iam.gserviceaccount.com'  # Can be found in the service account JSON key above
OAUTH_FLAG = 0 # 0 means we are using the service account credentials key for auth and 1 means we are using a refresh token 
CATALOG = 'arun-genai-bb'  # same as project name 
DRIVER_PATH_ABS = '/Library/simba/googlebigqueryodbc/lib/libgooglebigqueryodbc_sbu.dylib'
EMAIL = 'as-service@arun-genai-bb.iam.gserviceaccount.com'

#### Establish ODBC connection 

Option 1: Establishing connection using PyODBC directly

In [ ]:
connection_string = f'Driver={DRIVER_PATH_ABS};Catalog={CATALOG};KeyFilePath={SERVICE_ACCOUNT_KEY_PATH_ABS};OAuthMechanism={OAUTH_FLAG};Email={EMAIL}'
logger.info(connection_string)
connection = pyodbc.connect(connection_string)

Option 2: Using LangChain

In [ ]:
params = {
    'KeyFilePath': quote_plus(SERVICE_ACCOUNT_KEY_PATH_ABS),
    'Driver': quote_plus(DRIVER_PATH_ABS),
    'OAuthMechanism': OAUTH_FLAG,
    'Catalog': CATALOG,
    'Email': EMAIL,
}

SQLALCHEMY_URI = f"bigquery://{PROJECT}/{DATASET}?{'&'.join(f'{k}={v}' for k, v in params.items())}"
logger.info(f'SQLALCHEMY_URI={SQLALCHEMY_URI}')

In [ ]:
db = SQLDatabase.from_uri(SQLALCHEMY_URI)

In [ ]:
db.__dict__

List all installed drivers 